<a href="https://colab.research.google.com/github/AlexPazCodesUCSD/CSE151AGroupProject/blob/main/CSE151A_Group_Project_Amazon_Agent_Rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run to install all the necessary packages for the project

%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
##this is a test comment

# Install more packages using %pip as needed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 60.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 23.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.3 MB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 42.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.1/262.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 M

In [2]:
# Loading the necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic

# Add more packages as needed

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'geopy'

## Data Link
[Kaggle](https://www.kaggle.com/datasets/sujalsuthar/amazon-delivery-dataset)
[Github](https://github.com/AlexPazCodesUCSD/CSE151AGroupProject/blob/main/amazon_delivery.csv)

In [ ]:
# Data Download into Google Colab

!wget https://raw.githubusercontent.com/AlexPazCodesUCSD/CSE151AGroupProject/main/amazon_delivery.csv

# Run only once, you can see the file by clicking the file icon on the leftside of the colab notebook

In [2]:
# Loading Data

data = pd.read_csv('amazon_delivery.csv')

data.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


# Data Clean Up

In [3]:
# The initial dimensions of the dataset
initial_dimensions = data.shape

# Identify columns with missing values and count the number of missing values in each
missing_values = data.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0]

# Print out the columns with missing values and the count of missing values in each
print("Columns with missing values and the count of missing values:")
print(columns_with_missing_values)

# Cleaning the data by dropping rows with missing values
cleaned_data = data.dropna()

# The dimensions of the cleaned dataset
cleaned_dimensions = cleaned_data.shape

# Print out the dimensions
print(f"Initial Data Dimensions: # of observations = {initial_dimensions[0]}")
print(f"Cleaned Data Dimensions: # of observations = {cleaned_dimensions[0]}")

data = cleaned_data

Columns with missing values and the count of missing values:
Agent_Rating    54
Weather         91
dtype: int64
Initial Data Dimensions: # of observations = 43739
Cleaned Data Dimensions: # of observations = 43594


## Creating More Features \(Data Preprocessing\)

In [4]:
# Function to calculate distance in miles between store and drop-off locations
def calculate_distance(row):
    store_location = (row['Store_Latitude'], row['Store_Longitude'])
    drop_location = (row['Drop_Latitude'], row['Drop_Longitude'])
    return geodesic(store_location, drop_location).miles

# Function to determine the delivery season based on the month
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

# Convert Order_Date, Order_Time, and Pickup_Time to appropriate datetime formats
data['Order_Date'] = pd.to_datetime(data['Order_Date'], format='%Y-%m-%d')
data['Order_Time'] = pd.to_datetime(data['Order_Time'], format='%H:%M:%S').dt.time
data['Pickup_Time'] = pd.to_datetime(data['Pickup_Time'], format='%H:%M:%S').dt.time


data['Distance_Miles'] = data.apply(calculate_distance, axis=1)
data['Delivery_Season'] = data['Order_Date'].apply(get_season)

# Add more if desired

data.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,Distance_Miles,Delivery_Season
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing,1.876999,Spring
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics,12.516738,Spring
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports,0.962935,Spring
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics,4.830848,Spring
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys,3.851195,Spring


# Dataset Column Descriptions and Statistics

1. **Order_ID**: Unique identifier for each order.
2. **Agent_Age**: Age of the delivery agent.
3. **Agent_Rating**: Rating of the delivery agent.
4. **Store_Latitude**: Latitude of the store location.
5. **Store_Longitude**: Longitude of the store location.
6. **Drop_Latitude**: Latitude of the drop-off location.
7. **Drop_Longitude**: Longitude of the drop-off location.
8. **Order_Date**: Date of the order.
9. **Order_Time**: Time of the order.
10. **Pickup_Time**: Time the order was picked up.
11. **Weather**: Weather conditions during delivery.
12. **Traffic**: Traffic conditions during delivery.
13. **Vehicle**: Type of vehicle used for delivery.
14. **Area**: Area type (e.g., Urban, Metropolitan).
15. **Delivery_Time**: Time taken for delivery (in minutes).
16. **Category**: Category of the delivered item.
17. **Distance_Miles**: Distance in miles between the store and the drop-off location, calculated using latitude and longitude.
18. **Delivery_Season**: The season during which the delivery was made, derived from the order date.


In [5]:
# Function to analyze and provide information on each column
def analyze_column(data, column_name):
    col_data = data[column_name]
    col_info = {}

    # Check if the column is numerical
    if pd.api.types.is_numeric_dtype(col_data):
        if "latitude" in column_name.lower() or "longitude" in column_name.lower():
            col_info['type'] = 'Geographical coordinates'
            col_info['description'] = 'Latitude and longitude values should not be analyzed statistically.'
        else:
            col_info['type'] = 'Numerical'
            col_info['statistics'] = col_data.describe().to_dict()

    # Check if the column is categorical
    elif pd.api.types.is_string_dtype(col_data):
        if column_name.lower() == 'order_id':
            col_info['type'] = 'Unique identifier'
            col_info['description'] = 'Unique ID for each order.'
        else:
            col_info['type'] = 'Categorical'
            col_info['unique_values'] = col_data.nunique()
            col_info['categories'] = col_data.value_counts().to_dict()

    # Check if the column is datetime
    elif pd.api.types.is_datetime64_any_dtype(col_data):
        col_info['type'] = 'Datetime'
        col_info['description'] = 'Datetime values for orders and pickups.'

    # Check if the column is time
    elif pd.api.types.is_object_dtype(col_data) and column_name.lower().endswith('_time'):
        col_info['type'] = 'Time'
        col_info['description'] = 'Time values for orders and pickups.'

    else:
        col_info['type'] = 'Other'
        col_info['description'] = 'Other type of data.'

    return col_info

column_analysis = {}
for column in data.columns:
    column_analysis[column] = analyze_column(data, column)

for column, analysis in column_analysis.items():
    print(f"Column: {column}")
    for key, value in analysis.items():
        print(f"  {key}: {value}")
    print()


print("First few rows of the updated dataset:")
data.head()


Column: Order_ID
  type: Unique identifier
  description: Unique ID for each order.

Column: Agent_Age
  type: Numerical
  statistics: {'count': 43594.0, 'mean': 29.55571867688214, 'std': 5.760689023877086, 'min': 20.0, '25%': 25.0, '50%': 30.0, '75%': 35.0, 'max': 39.0}

Column: Agent_Rating
  type: Numerical
  statistics: {'count': 43594.0, 'mean': 4.635286966096252, 'std': 0.31382654114941966, 'min': 2.5, '25%': 4.5, '50%': 4.7, '75%': 4.9, 'max': 5.0}

Column: Store_Latitude
  type: Geographical coordinates
  description: Latitude and longitude values should not be analyzed statistically.

Column: Store_Longitude
  type: Geographical coordinates
  description: Latitude and longitude values should not be analyzed statistically.

Column: Drop_Latitude
  type: Geographical coordinates
  description: Latitude and longitude values should not be analyzed statistically.

Column: Drop_Longitude
  type: Geographical coordinates
  description: Latitude and longitude values should not be anal

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,Distance_Miles,Delivery_Season
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing,1.876999,Spring
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics,12.516738,Spring
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports,0.962935,Spring
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics,4.830848,Spring
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys,3.851195,Spring
